# Import Dependencies and Create DataBunch

In [0]:
#Training an IMDb Sentiment Model with ULMFiT ((Universal Language Model Fine-tuning) for Text Classification).

In [0]:
from fastai.text import *

In [3]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/root/.fastai/data/imdb_sample')

In [4]:
!ls {path}

texts.csv


In [5]:
df = pd.read_csv(path/'texts.csv')
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


In [0]:
data_lm = TextLMDataBunch.from_csv(path, 'texts.csv')
data_clas = TextClasDataBunch.from_csv(path, 'texts.csv', vocab = data_lm.train_ds.vocab, bs = 32)

In [12]:
data_lm.show_batch()

idx,text
0,"xxmaj the xxmaj crater xxmaj lake xxmaj monster "" amateurish is to xxunk the obvious . xxmaj if you are n't a fan of low budget drive - in films , you probably would n't be looking here in the first place . \n \n xxmaj the problem with the movie is that when there 's no monster action going on , it really sucks and goes nowhere ."
1,"was . xxmaj if you read that and thought that was pure comic brilliance ( sadly , it probably was the most xxup intentionally funny part of the movie ) , maybe you 'll like this movie . xxmaj but if you 're a xxmaj vonnegut fan or not completely insane , do n't see it . xxmaj please . xxbos xxmaj xxunk 3 is bad ... awfully bad"
2,"together and hang out with cool animals and stay away from cities . xxmaj no wonder they 're happy . xxbos i was not expecting much going in to this , but still came away disappointed . xxmaj this was my least favorite xxmaj halestorm production i have seen . i thought it was supposed to be a comedy , but i only xxunk at 3 or 4 jokes"
3,"or fighting is xxunk for acting ! ) ... what does that leave ? xxmaj yes the action scenes . xxmaj the action scenes are not bad and that 's the reason i gave the movie 2 stars instead of 1 ! i was giving this movie a chance , but it was a waste of time ... xxmaj you have better things to do / watch , believe"
4,". xxmaj alright , i did watch taxi , but i said penance and was xxunk for that mistake . xxmaj but , this movie had xxmaj drew xxmaj barrymore and i just think she is the xxunk thing . i do n't know what it is about her , but i just smile the whole time i am watch her . xxmaj she just xxunk me that way"


In [0]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [0]:
data_lm = load_data(path, 'data_lm_export.pkl')
data_clas = load_data(path, 'data_clas_export.pkl', bs = 16)

# Language Model Training, Testing and Interpretation

In [0]:
#Language Modeling is the task of Predicting the next word or character in a document.

In [15]:
data_lm.show_batch()

idx,text
0,"xxmaj the xxmaj crater xxmaj lake xxmaj monster "" amateurish is to xxunk the obvious . xxmaj if you are n't a fan of low budget drive - in films , you probably would n't be looking here in the first place . \n \n xxmaj the problem with the movie is that when there 's no monster action going on , it really sucks and goes nowhere ."
1,"was . xxmaj if you read that and thought that was pure comic brilliance ( sadly , it probably was the most xxup intentionally funny part of the movie ) , maybe you 'll like this movie . xxmaj but if you 're a xxmaj vonnegut fan or not completely insane , do n't see it . xxmaj please . xxbos xxmaj xxunk 3 is bad ... awfully bad"
2,"together and hang out with cool animals and stay away from cities . xxmaj no wonder they 're happy . xxbos i was not expecting much going in to this , but still came away disappointed . xxmaj this was my least favorite xxmaj halestorm production i have seen . i thought it was supposed to be a comedy , but i only xxunk at 3 or 4 jokes"
3,"or fighting is xxunk for acting ! ) ... what does that leave ? xxmaj yes the action scenes . xxmaj the action scenes are not bad and that 's the reason i gave the movie 2 stars instead of 1 ! i was giving this movie a chance , but it was a waste of time ... xxmaj you have better things to do / watch , believe"
4,". xxmaj alright , i did watch taxi , but i said penance and was xxunk for that mistake . xxmaj but , this movie had xxmaj drew xxmaj barrymore and i just think she is the xxunk thing . i do n't know what it is about her , but i just smile the whole time i am watch her . xxmaj she just xxunk me that way"


In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult = 0.5)

In [17]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.512565,3.957747,0.286719,12:41


In [0]:
learn.unfreeze()

In [19]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.171539,3.858399,0.293718,20:01


In [0]:
#That Accuracy is a good one as it is a language model.
#Our Pretrained Model is trained on WikiText-103 Dataset.
#WikiText-103 has the test perplexity = 17.48 using GPT-2
#     "       "       "       "       = 18.3 using Transformer-XL Large

# Prediction - Inference

In [20]:
learn.predict('Life is', n_words = 8)

'Life is ! i am desperate for a wife in'

In [0]:
learn.save_encoder('ft_enc')

# Classifier Training, Testing and Interpretation.

In [0]:
#Sentiment Analysis is the task of Classifying the Polarity of a given Text.

In [22]:
data_clas.show_batch()

text,target
"xxbos xxup the xxup shop xxup around xxup the xxup corner is one of the xxunk and most feel - good romantic comedies ever made . xxmaj there 's just no getting around that , and it 's hard to actually put one 's feeling for this film into words . xxmaj it 's not one of those films that tries too hard , nor does it come up with",positive
"xxbos xxmaj this film sat on my xxmaj tivo for weeks before i watched it . i dreaded a self - indulgent xxunk flick about relationships gone bad . i was wrong ; this was an xxunk xxunk into the xxunk - up xxunk of xxmaj new xxmaj yorkers . \n \n xxmaj the format is the same as xxmaj max xxmaj xxunk ' "" xxmaj la xxmaj ronde",positive
"xxbos i really wanted to love this show . i truly , honestly did . \n \n xxmaj for the first time , gay viewers get their own version of the "" xxmaj the xxmaj bachelor "" . xxmaj with the help of his obligatory "" hag "" xxmaj xxunk , xxmaj james , a good looking , well - to - do thirty - something has the chance",negative
"xxbos \n \n i 'm sure things did n't exactly go the same way in the real life of xxmaj homer xxmaj hickam as they did in the film adaptation of his book , xxmaj rocket xxmaj boys , but the movie "" xxmaj october xxmaj sky "" ( an xxunk of the book 's title ) is good enough to stand alone . i have not read xxmaj",positive
"xxbos xxmaj how viewers react to this new "" adaption "" of xxmaj shirley xxmaj jackson 's book , which was promoted as xxup not being a remake of the original 1963 movie ( true enough ) , will be based , i suspect , on the following : those who were big fans of either the book or original movie are not going to think much of this one",negative


In [0]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult = 0.5)

In [0]:
learn.load_encoder('ft_enc')

In [25]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.615350,0.546994,0.746269,14:25


In [0]:
learn.freeze_to(-2)

In [27]:
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.562568,0.484885,0.791045,19:42


In [0]:
learn.unfreeze()

In [29]:
learn.fit_one_cycle(1, slice(2e-3/100., 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.462699,0.407138,0.820895,54:22


In [30]:
learn.predict("This is good")

(Category positive, tensor(1), tensor([0.3451, 0.6549]))

In [0]:
learn.save('stage-1')

In [0]:
#We achieved an Accuracy of 82%.